In [36]:
import pandas
reviews = pandas.read_csv("data/denormalized-moderated-2021.csv")
reviews['label'] = reviews.apply(lambda row: 1 if row['Accept/Reject'] == 'Accept' else 0, axis=1)
field_separator = "\n"
reviews['full_text'] = reviews.apply(lambda row: f"Name: {row['screenname']}{field_separator}Title: {row['title']}{field_separator}Comments: {row['comments']}{field_separator}Exterior: {row['exterior_review']}{field_separator}Interior: {row['interior_review']}{field_separator}Family: {row['family_review']}{field_separator}Cost: {row['cost_review']}Performance: {field_separator}{row['perf_review']}", axis=1)
reviews[['label','full_text']].head()


,label,full_text
0,0,Name: Jose\nTitle: Great love it\nComments: I ...
1,0,Name: Subaru Fan\nTitle: Hardly been driven\nC...
2,1,Name: WDS\nTitle: Features\nComments: These ar...
3,1,Name: Nino\nTitle: Great car!\nComments: Great...
4,1,Name: BMW650IGranCoupe\nTitle: True performanc...


In [37]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = "true"

In [3]:
!pip install transformers[torch]

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 128.8 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 55.0 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.1/182.1 KB 38.0 MB/s eta 0:00:00
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p38/bin/python -m pip install --upgrade pip' command.


In [38]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

loading configuration file config.json from cache at /home/ec2-user/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.24.0",
  "vocab_size": 30522
}

loading file vocab.txt from cache at /home/ec2-user/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/vocab.txt
loading file tokenizer.json from cache at /home/ec2-user/.cache/huggingface/hub/models--dis

In [39]:
def preprocess_function(row):
    return tokenizer(str(row['full_text']), truncation=True, padding=True)

In [6]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
  Using cached datasets-2.7.0-py3-none-any.whl (451 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.9/212.9 KB 2.2 MB/s eta 0:00:00a 0:00:01
  Using cached responses-0.18.0-py3-none-any.whl (38 kB)
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/pytorch_p38/bin/python -m pip install --upgrade pip' command.


In [ ]:

# reviews = pandas.DataFrame(data={'full_text': ["great car", 
#                                         "fantastic ride",
#                                         "has some issues",
#                                        "best car ever",
#                                        "sucks blows horrible",
#                                        "mamba buahaha",
#                                        "good car",
#                                         "my best ride",
#                                         "your mama is so fat",
#                                         "bring back the manuals",
#                                         "honda rocks",
#                                         "my honda drives best after it had a few drinks",
#                                         "good bad and the ugly about this car",
#                                         "is not a good car",
#                                         "i like to read books and long walks on the beach",
#                                         "your butt smells"
#                                 ],
#                           'label': [1, 1,1,1,0,0,1,1,0,1,1,0,1,1,0,0]})

In [40]:
train = reviews.sample(frac=0.99, random_state= 25)
test = reviews.drop(train.index)

from datasets import Dataset
train_ds = Dataset.from_pandas(train[['full_text','label']])
test_ds = Dataset.from_pandas(test[['full_text','label']])

tokenized_train_ds = train_ds.map(preprocess_function)
tokenized_test_ds = test_ds.map(preprocess_function)

  0%|          | 0/122348 [00:00<?, ?ex/s]

  0%|          | 0/1236 [00:00<?, ?ex/s]

In [41]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [42]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels = 2)

loading configuration file config.json from cache at /home/ec2-user/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.24.0",
  "vocab_size": 30522
}

loading weights file pytorch_model.bin from cache at /home/ec2-user/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/pytorch_model.bin
Some weights of the model checkpoint at distilbert-base-uncased wer

In [43]:
training_args = TrainingArguments(
    output_dir="./results3",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=25,
    weight_decay=0.01,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_ds,
    eval_dataset=tokenized_test_ds,
    tokenizer=tokenizer,
    data_collator=data_collator,
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [47]:
trainer.train()

/home/ec2-user/anaconda3/envs/pytorch_p38/lib/python3.8/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 122348
  Num Epochs = 25
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 191175
  Number of trainable parameters = 66955010
The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: full_text, __index_level_0__. If full_text, __index_level_0__ are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
You're using a DistilBertTokenizerFast tokeni

Step,Training Loss
500,0.333100
1000,0.324700
1500,0.317100
2000,0.305900
2500,0.307700
3000,0.303500
3500,0.317200
4000,0.290300
4500,0.297500
5000,0.296700


Saving model checkpoint to ./results3/checkpoint-500
Configuration saved in ./results3/checkpoint-500/config.json
Model weights saved in ./results3/checkpoint-500/pytorch_model.bin
tokenizer config file saved in ./results3/checkpoint-500/tokenizer_config.json
Special tokens file saved in ./results3/checkpoint-500/special_tokens_map.json
Saving model checkpoint to ./results3/checkpoint-1000
Configuration saved in ./results3/checkpoint-1000/config.json
Model weights saved in ./results3/checkpoint-1000/pytorch_model.bin
tokenizer config file saved in ./results3/checkpoint-1000/tokenizer_config.json
Special tokens file saved in ./results3/checkpoint-1000/special_tokens_map.json
Saving model checkpoint to ./results3/checkpoint-1500
Configuration saved in ./results3/checkpoint-1500/config.json
Model weights saved in ./results3/checkpoint-1500/pytorch_model.bin
tokenizer config file saved in ./results3/checkpoint-1500/tokenizer_config.json
Special tokens file saved in ./results3/checkpoint-15

TrainOutput(global_step=191175, training_loss=0.0764588642165131, metrics={'train_runtime': 20731.2756, 'train_samples_per_second': 147.54, 'train_steps_per_second': 9.222, 'total_flos': 2.1869936378119037e+17, 'train_loss': 0.0764588642165131, 'epoch': 25.0})

In [44]:
import torch
torch.cuda.empty_cache()

#print(torch.cuda.is_available())
#torch.cuda.device_count()
#torch.cuda.get_device_name(0)

In [45]:
reviews = None
test = None
train = None
train_ds = None
test_ds = None

import gc
gc.collect()

34

In [46]:
!nvidia-smi

Tue Nov 22 02:51:05 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A10G         On   | 00000000:00:1E.0 Off |                    0 |
|  0%   20C    P0    58W / 300W |   3401MiB / 23028MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------